In [21]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from joblib import dump,load
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

credentials_path = r"service_account_json_file_path"
credentials = service_account.Credentials.from_service_account_file(credentials_path)
project_id = "bq_project_id"

sql = """

SELECT agg.*,val.* EXCEPT (user_pseudo_id), ltv90.itemRevenue90 FROM

(select * from 
(SELECT 
user_pseudo_id,
traffic_source.medium,
traffic_source.name,
traffic_source.source,
device.mobile_brand_name,
geo.country,
device.category,
row_number() OVER (PARTITION BY user_pseudo_id order by event_timestamp asc) as cnt
FROM 
--- Update the below dataset to match your GA4 dataset and project
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` LEFT JOIN UNNEST(items) AS items
WHERE _table_suffix >= '20201101' AND _table_suffix <= '20210131'
AND DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) BETWEEN '2020-11-01' AND '2020-11-30'
AND DATE_DIFF(DATE(TIMESTAMP_MICROS(event_timestamp)) , DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) , DAY) <=15)
WHERE cnt = 1) as agg

LEFT JOIN

(SELECT 
date(timestamp_micros(user_first_touch_timestamp)) as fdate,
user_pseudo_id,
--device.category,
--device.mobile_brand_name,
--geo.country,
--geo.city,
--geo.region,
--traffic_source.medium,
--traffic_source.name,
--traffic_source.source,
count(distinct (select value.int_value from unnest (event_params) WHERE KEY = 'ga_session_id')) as sessionCnt,
count(distinct case when event_name = 'session_start' THEN date(timestamp_micros(event_timestamp))END) as sessionDate,
count(distinct item_brand) itemBrandCount,
count(distinct item_category) itemCatCount,
count(distinct case when event_name = 'view_promotion' THEN event_timestamp END) as viwePromotion,
count(distinct case when event_name = 'select_promotion' THEN event_timestamp END) as SelectPromotion,


COUNT(DISTINCT case when event_name = 'view_item' THEN item_name END) AS itemViewCnt,
COUNT(DISTINCT case when event_name = 'select_item' THEN item_name END) AS itemSelectCnt,

count(distinct case when event_name = 'add_payment_info' THEN event_timestamp END) as paymetInfoAdd,
count(distinct case when event_name = 'add_shipping_info' THEN event_timestamp END) as shippingInfoAdd,


count(distinct CASE WHEN  event_name = 'scroll' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_location') END) ScrollpageLocationCnt,
count(distinct CASE WHEN  event_name = 'scroll' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_title') END) ScrollpageTitleCnt,

count(distinct CASE WHEN  event_name = 'page_view' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_location') END) pageViewPageLocationCnt,
count(distinct CASE WHEN  event_name = 'page_view' THEN (select value.string_value from unnest (event_params) WHERE KEY = 'page_title') END) pageViewPageTitleCnt,

--Item views
COUNT(distinct CASE WHEN event_name = 'view_item' THEN event_timestamp ELSE NULL END) AS itemViews,
--Add-to-carts
COUNT( DISTINCT CASE WHEN event_name = 'add_to_cart' THEN event_timestamp ELSE NULL END) AS addToCarts,
COUNT( DISTINCT CASE WHEN event_name = 'add_to_cart' THEN items.item_id ELSE NULL END) AS addToItemId,

COUNT( DISTINCT CASE WHEN event_name = 'view_search_results' THEN event_timestamp ELSE NULL END) AS searchResultViewedCnt,
count(distinct case when event_name = 'begin_checkout' THEN event_timestamp else null end) as checkOut,
--Ecommerce Purchases
COUNT(CASE WHEN event_name = 'purchase' THEN ecommerce.transaction_id ELSE NULL END) AS ecommercePurchases, 
--Purchase-to-view-rate
(CASE WHEN COUNT(CASE WHEN event_name = 'view_item' THEN  event_timestamp ELSE NULL END) = 0 THEN 0
ELSE COUNT(DISTINCT CASE WHEN event_name = 'purchase' THEN event_timestamp  ELSE NULL END) /
COUNT(DISTINCT CASE WHEN event_name = 'view_item' THEN event_timestamp  ELSE NULL END) END  * 100) AS purchaseToViewRate,
--Item purchase quantity
COUNT(distinct CASE WHEN event_name = 'purchase' THEN items.item_name  ELSE NULL END) AS itemPurchaseName,

IFNULL(SUM(CASE WHEN event_name = 'purchase' THEN items.quantity  ELSE NULL END),0) AS itemPurchaseQuantity,
--Item revenue
IFNULL(SUM(item_revenue),0) AS itemRevenue15,
IFNULL(SUM(CASE WHEN event_name = 'purchase' THEN item_revenue ELSE NULL END),0) AS itemRevenue2

FROM 
--- Update the below dataset to match your GA4 dataset and project
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` LEFT JOIN UNNEST(items) AS items
WHERE _table_suffix >= '20201101' AND _table_suffix <= '20210131'
AND DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) BETWEEN '2020-11-01' AND '2020-11-30'
AND DATE_DIFF(DATE(TIMESTAMP_MICROS(event_timestamp)) , DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) , DAY) <=15
GROUP BY 1,2
) as val on agg.user_pseudo_id = val.user_pseudo_id

LEFT JOIN

(SELECT 
date(timestamp_micros(user_first_touch_timestamp)) as fdate,
user_pseudo_id,

IFNULL(SUM(item_revenue),0) AS itemRevenue90,

FROM 
--- Update the below dataset to match your GA4 dataset and project
`bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` LEFT JOIN UNNEST(items) AS items
WHERE _table_suffix >= '20201101' AND _table_suffix <= '20210131'
AND DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) BETWEEN '2020-11-01' AND '2020-11-30'
AND DATE_DIFF(DATE(TIMESTAMP_MICROS(event_timestamp)) , DATE(TIMESTAMP_MICROS(user_first_touch_timestamp)) , DAY) <=90
GROUP BY 1,2
) as ltv90 on agg.user_pseudo_id = ltv90.user_pseudo_id


"""

In [22]:
##df = pandas_gbq.read_gbq(sql, project_id= project_id)
df = pd.read_pickle("df.pkl")

In [23]:
gdp = pd.read_pickle("gdp.pkl")
gdp

,"GDP per capita, current prices\n (U.S. dollars per capita)",1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
0,Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,233.433,233.755,254.259,294.396,320.674,381.502,447.746,511.374,631.49,714.7,784.611,754.402,746.922,705.597,617.126,635.789,580.253,586.204,611.268,423.993,422.229,no data,no data,no data,no data,no data,no data,no data
1,Albania,728.359,817.734,824.542,815.529,788.332,788.801,855.724,832.245,805.046,860.784,675.736,408.022,259.75,452.623,736.028,903.989,1009.967,717.381,818.32,1032.264,1127.64,1283.573,1425.125,1846.119,2373.58,2673.773,2972.745,3595.053,4370.562,4114.09,4097.833,4439.89,4248.909,4415.6,4584.919,3953.611,4124.405,4525.887,5257.714,5345.058,5278.986,6259.762,6657.637,7956.559,8924.317,9474.065,10135.055,10827.923,11578.584,12389.74
2,Algeria,2462.085,2502.13,2446.588,2514.256,2640.191,2988.545,2929.092,2935.816,2326.571,2309.34,2684.465,1975.201,2033.196,2056.556,1674.567,1626.997,1783.41,1800.196,1772.368,1769.091,1947.755,1919.41,1958.238,2304.603,2839.82,3258.153,3690.572,4208.244,5214.762,4262.144,4941.493,5946.345,6057.974,5997.896,6108.878,4691.686,4426.58,4554.15,4568.244,4452.856,3757.585,4169.984,4981.822,5323.635,5721.678,5869.435,5998.622,6073.858,6129.969,6170.756
3,Andorra,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,49030.229,51957.313,45713.97,45630.195,46299.282,38877.485,39595.317,40017.741,42229.903,40688.491,36973.845,41806.876,41378.881,43784.571,44899.596,45570.14,46093.258,46597.511,47098.423,47563.642
4,Angola,802.627,731.427,712.576,723.654,747.325,817.068,743.735,828.958,875.526,991.295,1062.788,996.472,743.754,522.991,368.84,445.424,555.75,631.28,517.868,474.186,681.129,645.145,872.658,982.806,1254.696,1900.724,2597.964,3121.349,4081.718,3123.699,3586.664,4608.155,5083.827,5228.511,5371.217,4130.93,3468.518,4039.3,3240.858,2612.246,1709.283,2169.648,3439.391,2565.912,2431.58,2444.365,2455.711,2528.702,2571.26,2617.815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Latin America and the Caribbean,2536.298,2684.736,2147.602,1908.459,1947.855,2052.065,1986.52,2075.082,2300.953,2537.702,2800.869,2969.507,3198.326,3547.895,3935.574,4144.077,4407.078,4751.492,4725.487,4187.39,4517.817,4350.561,3808.849,3820.08,4320.569,5177.677,5999.654,7025.198,8074.088,7364.895,9059.053,10350.268,10408.843,10426.828,10224.111,8573.09,8178.796,9018.768,8730.494,8469.845,7043.843,8140.639,9241.181,10291.151,10878.689,11242.058,11759.286,12286.797,12788.801,13301.689
223,Major advanced economies (G7),11275.803,11554.398,11436.111,12005.038,12568.159,13252.936,15946.884,18227.138,20367.02,20993.818,23018.82,24446.4,26047.098,26370.878,28017.777,30368.172,30128.049,29551.161,29766.457,31199.193,31750.15,31188.325,31964.003,35077.208,38308.926,39753.014,41218.282,44012.765,45895.267,43468.804,44925.825,47524.06,47534.301,47381.427,48312.351,46172.726,47246.175,48718.129,51211.676,51968.616,50839.174,55977.766,57110.146,60221.173,62412.739,64658.942,67067.991,69485.17,72050.992,74590.436
224,Middle East and Central Asia,2108.544,2179.306,2101.667,2075.924,2039.69,2041.836,2023.437,2329.576,2385.981,2653.528,3170.981,2237.338,1364.372,1464.874,1515.703,1683.444,1874.559,1946.325,1912.958,2099.505,2418.101,2289.981,1839.978,1993.239,2340.994,2788.01,3253.49,3804.817,4

In [24]:
gdp = gdp[['GDP per capita, current prices\n (U.S. dollars per capita)','2020','2021']]
gdp = gdp.rename(columns= {'GDP per capita, current prices\n (U.S. dollars per capita)':'country'})
gdp = gdp.rename(columns= {'2020':'gdp_2020_value'})
gdp = gdp.rename(columns= {'2021':'gdp_2021_value'})

In [25]:
merged_df = pd.merge(df, gdp, on='country',  how='left')
merged_df

,user_pseudo_id,medium,name,source,mobile_brand_name,country,category,cnt,fdate,sessionCnt,sessionDate,itemBrandCount,itemCatCount,viwePromotion,SelectPromotion,itemViewCnt,itemSelectCnt,paymetInfoAdd,shippingInfoAdd,ScrollpageLocationCnt,ScrollpageTitleCnt,pageViewPageLocationCnt,pageViewPageTitleCnt,itemViews,addToCarts,addToItemId,searchResultViewedCnt,checkOut,ecommercePurchases,purchaseToViewRate,itemPurchaseName,itemPurchaseQuantity,itemRevenue15,itemRevenue2,itemRevenue90,gdp_2020_value,gdp_2021_value
0,1079995.2140121274,organic,(organic),google,Apple,India,desktop,1,2020-11-27,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,1915.552,2250.179
1,1087261.5318242318,organic,(organic),<Other>,Apple,United States,desktop,1,2020-11-27,1,1,2,7,1,0,46,0,0,0,4,4,6,6,7,6,44,0,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
2,1088915.9629798165,(none),(direct),(direct),Microsoft,United States,desktop,1,2020-11-20,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
3,10958150.6125765638,organic,(organic),google,Apple,United States,desktop,1,2020-11-19,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
4,11054427.9320970826,(none),(direct),(direct),<Other>,United States,desktop,1,2020-11-17,1,1,2,2,3,0,11,0,0,2,7,6,9,7,2,1,12,0,4,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71733,9420210.3127123637,organic,(organic),google,<Other>,United States,mobile,1,2020-11-23,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794
71734,95956898.6638871488,organic,(organic),google,Google,India,desktop,1,2020-11-04,2,1,1,1,2,0,0,0,0,0,2,2,2,2,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,1915.552,2250.179
71735,9611368.8927276296,organic,(organic),google,Apple,Greece,mobile,1,2020-11-23,2,2,1,1,3,0,0,0,0,0,2,2,2,2,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,17570.395,20115.924
71736,99175181.8355789716,(none),(direct),(direct),Apple,United States,desktop,1,2020-11-03,2,1,1,1,1,0,0,0,0,0,1,1,2,2,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0.0,64367.435,70995.794


In [26]:
merged_df['gdp_2020_value'] = merged_df['gdp_2020_value'].astype(float)
merged_df['gdp_2021_value'] = merged_df['gdp_2021_value'].astype(float)
merged_df['Avg_gdp'] =  merged_df[['gdp_2020_value','gdp_2021_value']].mean(axis = 1)

In [27]:
country_filtered = merged_df[merged_df['country'].isin(["United States", "Canada", "Germany", "India", "France", "Taiwan",'Italy', "Japan","Spain", "China", "Singapore", "South Korea", "Netherlands", 'Turkey'])]

In [28]:
country_filtered.loc[:, 'itemRevenue90'] = country_filtered['itemRevenue90'].apply(lambda x: x if x > 0 else 1e-10)
country_filtered.loc[:, 'LogLtv90'] = np.log(country_filtered['itemRevenue90'])
country_filtered.loc[:, 'LogGDP'] = np.log(country_filtered['Avg_gdp'])

C:\Users\w11\AppData\Local\Temp\ipykernel_15660\1260032352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_filtered.loc[:, 'LogLtv90'] = np.log(country_filtered['itemRevenue90'])
C:\Users\w11\AppData\Local\Temp\ipykernel_15660\1260032352.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_filtered.loc[:, 'LogGDP'] = np.log(country_filtered['Avg_gdp'])


In [29]:
dummies_df = pd.get_dummies(country_filtered[['medium', 'mobile_brand_name', 'country', 'category']], drop_first=True, dtype=int)
dummy_df = pd.concat([country_filtered,dummies_df], axis = 1)


In [30]:
x = dummy_df.drop(['itemRevenue2','itemRevenue90', 'LogLtv90', 'user_pseudo_id','medium','name','source','mobile_brand_name','country','category','cnt','fdate'], axis = 1)
y = (dummy_df.LogLtv90)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [32]:
pd.set_option('display.max_columns', None)
X_train

,sessionCnt,sessionDate,itemBrandCount,itemCatCount,viwePromotion,SelectPromotion,itemViewCnt,itemSelectCnt,paymetInfoAdd,shippingInfoAdd,ScrollpageLocationCnt,ScrollpageTitleCnt,pageViewPageLocationCnt,pageViewPageTitleCnt,itemViews,addToCarts,addToItemId,searchResultViewedCnt,checkOut,ecommercePurchases,purchaseToViewRate,itemPurchaseName,itemPurchaseQuantity,itemRevenue15,gdp_2020_value,gdp_2021_value,Avg_gdp,LogGDP,medium_(none),medium_<Other>,medium_cpc,medium_organic,medium_referral,mobile_brand_name_Apple,mobile_brand_name_Google,mobile_brand_name_Huawei,mobile_brand_name_Microsoft,mobile_brand_name_Mozilla,mobile_brand_name_Samsung,mobile_brand_name_Xiaomi,country_China,country_France,country_Germany,country_India,country_Italy,country_Japan,country_Netherlands,country_Singapore,country_South Korea,country_Spain,country_Taiwan,country_Turkey,country_United States,category_mobile,category_tablet
41219,1,1,2,1,1,0,11,0,0,0,1,1,3,3,1,0,0,0,0,0,0.0,0,0,0.0,1915.552,2250.179,2082.8655,7.641500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
11161,6,3,2,2,3,1,12,0,0,0,0,0,3,3,1,0,0,0,0,0,0.0,0,0,0.0,1915.552,2250.179,2082.8655,7.641500,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
53549,1,1,1,1,3,0,0,0,0,0,2,2,2,2,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
49220,2,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,1915.552,2250.179,2082.8655,7.641500,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
62858,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0,0,0.0,8612.903,9663.877,9138.3900,9.120240,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57919,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
70411,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
49483,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1096,2,1,2,2,1,0,11,0,0,0,2,2,3,3,2,0,0,0,0,0,0.0,0,0,0.0,40171.929,40114.287,40143.1080,10.600206,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [33]:
X_test

,sessionCnt,sessionDate,itemBrandCount,itemCatCount,viwePromotion,SelectPromotion,itemViewCnt,itemSelectCnt,paymetInfoAdd,shippingInfoAdd,ScrollpageLocationCnt,ScrollpageTitleCnt,pageViewPageLocationCnt,pageViewPageTitleCnt,itemViews,addToCarts,addToItemId,searchResultViewedCnt,checkOut,ecommercePurchases,purchaseToViewRate,itemPurchaseName,itemPurchaseQuantity,itemRevenue15,gdp_2020_value,gdp_2021_value,Avg_gdp,LogGDP,medium_(none),medium_<Other>,medium_cpc,medium_organic,medium_referral,mobile_brand_name_Apple,mobile_brand_name_Google,mobile_brand_name_Huawei,mobile_brand_name_Microsoft,mobile_brand_name_Mozilla,mobile_brand_name_Samsung,mobile_brand_name_Xiaomi,country_China,country_France,country_Germany,country_India,country_Italy,country_Japan,country_Netherlands,country_Singapore,country_South Korea,country_Spain,country_Taiwan,country_Turkey,country_United States,category_mobile,category_tablet
54173,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0,0,0.0,1915.552,2250.179,2082.8655,7.641500,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
63975,1,1,1,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33570,3,3,1,1,2,0,0,0,0,0,4,3,5,4,0,0,0,0,0,0,0.0,0,0,0.0,40529.093,45161.226,42845.1595,10.665348,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
356,2,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
44609,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52647,1,1,2,1,0,0,11,0,0,0,0,0,1,1,1,0,0,0,0,0,0.0,0,0,0.0,10525.001,12572.071,11548.5360,9.354314,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29226,5,1,2,6,4,0,37,11,0,0,10,10,17,16,10,2,2,4,0,0,0.0,0,0,0.0,26968.374,30563.552,28765.9630,10.266948,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
57020,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
52995,1,1,2,2,1,0,11,0,0,0,2,2,2,2,1,1,12,0,0,0,0.0,0,0,0.0,64367.435,70995.794,67681.6145,11.122570,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [34]:
y_train

41219   -23.025851
11161   -23.025851
53549   -23.025851
49220   -23.025851
62858   -23.025851
           ...    
57919   -23.025851
70411   -23.025851
49483   -23.025851
1096    -23.025851
20465   -23.025851
Name: LogLtv90, Length: 44290, dtype: float64

In [35]:
y_test

54173   -23.025851
63975   -23.025851
33570   -23.025851
356     -23.025851
44609   -23.025851
           ...    
52647   -23.025851
29226   -23.025851
57020   -23.025851
52995   -23.025851
5714    -23.025851
Name: LogLtv90, Length: 11073, dtype: float64

In [36]:
# Modeli başlatma ve eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # 100 ağaç kullanıyoruz
rf_model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred_rf = rf_model.predict(X_test)

# Performans metriklerini hesaplama
test_score_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"Test R2 Score: {test_score_rf:.3f}")
print(f"Mean Squared Error: {mse_rf:.3f}")
print(f"Root Mean Squared Error: {rmse_rf:.3f}")

Test R2 Score: 0.815
Mean Squared Error: 2.498
Root Mean Squared Error: 1.580


In [37]:
dump(rf_model, 'randomForest100Model_new.pkl')

['randomForest100Model_new.pkl']

In [38]:
import pickle
with open("randomForest100Model_new.pkl", "wb") as file:
    pickle.dump(rf_model, file)

In [39]:

with open("randomForest100Model_new.pkl", "rb") as file:
    model = pickle.load(file)

print(type(model))


<class 'sklearn.ensemble._forest.RandomForestRegressor'>
